In [6]:
import os
import numpy as np
import torch
import mdtraj as md
import torch.nn as nn
import torch.nn.functional as F
import deeptime
from torch.utils.data import DataLoader
from tqdm import tqdm
from deeptime.decomposition.deep import VAMPNet
from deeptime.decomposition import VAMP
from copy import deepcopy

class partitioned_fc(nn.Module):
    def __init__(self):
        super().__init__()
        self.inter = nn.Sequential(
            nn.Linear(20,30),
            nn.ReLU(inplace=True),
            nn.Linear(30,50),
            nn.ReLU(inplace=True),
            nn.Linear(50,80),
            nn.ReLU(inplace=True),
            nn.Linear(80,100),
            nn.ReLU(inplace=True),
        )
        self.intra = nn.Sequential(
            nn.Linear(190,170),
            nn.ReLU(inplace=True),
            nn.Linear(170,150),
            nn.ReLU(inplace=True),
            nn.Linear(150,130),
            nn.ReLU(inplace=True),
            nn.Linear(130,100),
            nn.ReLU(inplace=True),
        )
        self.com = 200
        #create a full connected MLP for combined vector and image data and output softmax
        self.combined_features = nn.Sequential(
            nn.Linear(self.com, int(self.com*(9/10))),
            nn.ReLU(inplace=True),
            nn.Linear(int(self.com*(9/10)),int(self.com*(8/10))),
            nn.ReLU(inplace = True),
            nn.Linear(int(self.com*(8/10)),int(self.com*(7/10))),
            nn.ReLU(inplace=True),
            nn.Linear(int(self.com*(7/10)),int(self.com*(5/10))),
            nn.ReLU(inplace=True),
            nn.Linear(int(self.com*(5/10)),int(self.com*(4/10))),
            nn.ReLU(inplace=True),
            nn.Linear(int(self.com*(4/10)),int(self.com*(3/10))),
            nn.Softmax(dim=1),
        )
    def forward(self,x):
        inter_ = self.inter(x[:,:20])
        intra_ = self.intra(x[:,20:])
        combined = torch.cat((inter_,intra_), 1)
        combined = self.combined_features(combined)
        return combined


In [2]:
import numpy as np
np.random.normal(5,5)

6.644594392669592

In [20]:
test= np.random.rand((10*210)).reshape(10,210)

In [21]:
net = partitioned_fc()

In [22]:
net(torch.from_numpy(test).float())

tensor([[0.0165, 0.0172, 0.0152, 0.0157, 0.0168, 0.0160, 0.0180, 0.0157, 0.0166,
         0.0163, 0.0159, 0.0184, 0.0150, 0.0181, 0.0152, 0.0183, 0.0190, 0.0179,
         0.0171, 0.0164, 0.0176, 0.0169, 0.0159, 0.0146, 0.0159, 0.0157, 0.0180,
         0.0165, 0.0158, 0.0165, 0.0174, 0.0172, 0.0168, 0.0161, 0.0178, 0.0178,
         0.0173, 0.0176, 0.0190, 0.0168, 0.0148, 0.0158, 0.0157, 0.0176, 0.0182,
         0.0155, 0.0158, 0.0159, 0.0166, 0.0166, 0.0161, 0.0176, 0.0161, 0.0156,
         0.0177, 0.0162, 0.0157, 0.0172, 0.0167, 0.0162],
        [0.0165, 0.0172, 0.0152, 0.0157, 0.0168, 0.0160, 0.0180, 0.0157, 0.0166,
         0.0163, 0.0159, 0.0183, 0.0150, 0.0181, 0.0152, 0.0183, 0.0190, 0.0179,
         0.0171, 0.0164, 0.0176, 0.0169, 0.0159, 0.0146, 0.0159, 0.0157, 0.0180,
         0.0165, 0.0158, 0.0165, 0.0174, 0.0172, 0.0168, 0.0161, 0.0178, 0.0178,
         0.0173, 0.0176, 0.0190, 0.0168, 0.0148, 0.0158, 0.0157, 0.0176, 0.0182,
         0.0155, 0.0158, 0.0159, 0.0166, 0.0166, 0.

In [23]:
!pwd


/home/t/msm/ntxd/VAMP/writhe/asyn/lig


In [ ]:
note_list = [cantus_firmus[i].transpose(interval) for interval in VALID_INTERVAL] #with list comp
midi_list = np.array(list(map(lambda note:note.pitch.midi, note_list))) #with map and lambda func

#...note_choice shit...

midi_dist = midi_list-int(note_choice) # we can choose to take the absolute value here but I think not
dist_idx = np.argsort(midi_dist)#smallest dist to farthest dist indices relative to note list
dist_sort = midi_dist[dist_idx]
note_sort = [note_list[idx] for idx in dist_idx]


## what if we wanted to condition on a certain distance?
constrained_idx = dist_idx[np.where(dist_sort%12 != 0 and ... and)] #we index dist_idx to maintain a map to the indices of

## so the closest one can be defined by all of the following
min_note = note_list[dist_idx[0]]; min_note = note_list[midi_dist.argmin()]; min_note = note_sort[0]

## now we can also define whether our note choice was up or down
direction = np.sign(dist_sort[constrained_idx[0]]) #we'll just take the sign of this boy
